# Project 5: Prompting With Large Language Models

In this project, we learn how to solve tasks by prompting existing LLM APIs. We will experiment with zero-shot and few-shot prompting and different ways methods for example selection for a semantic parsing task.

First we install and import the required dependencies. These include:
* `openai` as our API for querying LLMs (you are free to choose to use a different LLM API if you would like)


In [1]:
%%capture
%pip install openai


If you are using the OpenAI API, then go to then create an account and then copy your secret API key from `https://platform.openai.com/account/api-keys`. Set this as an environment variable or key management service so we can load it below. Make to keep private key secret. You may use a different LLM service if you choose to eg. Cohere (https://cohere.ai/).

In [11]:
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

If you have successfully authorized, then you should be able to see a list of available models by running the command below.

In [12]:
import openai
openai.api_key = OPENAI_API_KEY
openai.Model.list()

<OpenAIObject list at 0x10ac02860> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampl

We will now evaluate the LLM on a semantic parsing task. Geoquery is a dataset that contains information about the geography of the United States. For more information, please see: https://www.cs.utexas.edu/users/ml/nldata/geoquery.html. We will experiment with the compositional split introduced in (Keysers et al., 2020) https://openreview.net/forum?id=SygcCnNKwr. First, let's download the train and validation data. The goal for the LLM is to take in English queries about US geography about population, elevation, etc. and output a formal representation of the query.

In [13]:
!wget https://github.com/kl2806/geoquery/raw/main/data.zip -O data.zip
!unzip -o data.zip

--2023-04-27 21:04:55--  https://github.com/kl2806/geoquery/raw/main/data.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kl2806/geoquery/main/data.zip [following]
--2023-04-27 21:04:57--  https://raw.githubusercontent.com/kl2806/geoquery/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4493 (4.4K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   4.39K  --.-KB/s    in 0.001s  

2023-04-27 21:04:58 (8.55 MB/s) - ‘data.zip’ saved [4493/4493]

Archive:  data.zip
  inflating: train.tsv               
  inflating: dev.tsv      

Let's take a look at the data, each instance should compose of an English utterance, and a formal representation of the utterance

In [14]:
!head -n 5 ./train.tsv

how tall is the highest point in m0	answer ( elevation_1 ( highest ( intersection ( place , loc_2 ( m0 ) ) ) ) )
what is the largest city in m0	answer ( largest ( intersection ( city , loc_2 ( m0 ) ) ) )
what states border states that the m0 runs through	answer ( intersection ( state , next_to_2 ( intersection ( state , traverse_1 ( m0 ) ) ) ) )
what is the maximum elevation of m0	answer ( highest ( intersection ( place , loc_2 ( m0 ) ) ) )
what is the population of m0	answer ( population_1 ( m0 ) )


In [78]:
import csv
from dataclasses import dataclass
from typing import List

@dataclass
class Example:
     query: str
     program: str

training_examples: List[Example] = []

with open('./train.tsv', 'r') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for query, program in reader:
        training_examples.append(Example(query, program))

dev_examples: List[Example] = []
with open('./dev.tsv', 'r') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for query, program in reader:
        dev_examples.append(Example(query, program))

print(f"Num training examples: {len(training_examples)}")
print(f"Num dev examples: {len(dev_examples)}")


Num training examples: 440
Num dev examples: 40


Now, let's define a function that uses the OpenAI API to output the semantic parse. As a first cut, let's just try to describe the task in English the model and return it in `get_static_prompt`. Then implement `parse_example`, which should call the LLM API, and return the semantic parse. `gpt-3.5-turbo`, the corresponding API call to ChatGPT should work well for this task.

In [38]:
from typing import Callable

def get_static_prompt(utterance: str) -> str:
    return f"Translate the following into a semantic parse: {utterance}"

def parse_example(model: str, utterance: str, prompt_method: Callable[[str], str], **kwargs: dict) -> str:
    prompt = prompt_method(utterance, **kwargs)
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": """You are a helpful assistant."""},
            {"role": "user", "content": prompt}
        ],
        max_tokens=64,

        stop=["\n"]
    )

    return response.choices[0]


parse = parse_example(model="gpt-3.5-turbo",
                      utterance="what river runs through m0",
                      prompt_method=get_static_prompt)

print(parse)


{
  "finish_reason": "stop",
  "index": 0,
  "message": {
    "content": "[Ask] [what] [river] [runs through] [m0]",
    "role": "assistant"
  }
}


With just an English description, the output probably does not look very similar to the target language that we want. Let's try to construct a prompt with some examples from our training set and see how it does. Implement the function below for uniformly sampling examples from the training set, and using that to as a few-shot prompt to the model.

In [36]:
import random
from typing import List

def random_sample_prompt(utterance: str, training_examples: List[Example], num_samples: int = 10) -> str:
    """Return a prompt for a given example"""
    samples = random.sample(training_examples, min(num_samples, len(training_examples)))
    prompt = "Given the following examples of utterances and their semantic parses:\n"
    
    for example in samples:
        prompt += f"- Utterance: {example.query}\n  Parse: {example.program}\n"
    
    prompt += f"\nTranslate the following into a semantic parse: {utterance}"
    return prompt

    


prompt = random_sample_prompt(utterance="what river runs through m0",
                              training_examples=training_examples)
print("Uniform random sampling prompt: \n", prompt)


Uniform random sampling prompt: 
 Given the following examples of utterances and their semantic parses:
- Utterance: what are the populations of the major cities of m0
  Parse: answer ( population_1 ( intersection ( major , intersection ( city , loc_2 ( m0 ) ) ) ) )
- Utterance: what is the capital of the state m0
  Parse: answer ( intersection ( capital , loc_2 ( intersection ( state , m0 ) ) ) )
- Utterance: how many people lived in m0
  Parse: answer ( population_1 ( m0 ) )
- Utterance: what are the highest points of all the states
  Parse: answer ( highest ( intersection ( place , loc_2 ( state ) ) ) )
- Utterance: what is the largest city in m0
  Parse: answer ( largest ( intersection ( city , loc_2 ( m0 ) ) ) )
- Utterance: what rivers are in m0
  Parse: answer ( intersection ( river , loc_2 ( m0 ) ) )
- Utterance: what states border m0
  Parse: answer ( intersection ( state , next_to_2 ( m0 ) ) )
- Utterance: how many people live in the state with the largest population density


In [39]:
parse = parse_example(model="gpt-3.5-turbo",
                      utterance="what river runs through m0",
                      prompt_method=random_sample_prompt,
                      training_examples=training_examples)
print(parse)

{
  "finish_reason": "stop",
  "index": 0,
  "message": {
    "content": "Parse: answer ( intersection ( river , loc_2 ( m0 ) ) )",
    "role": "assistant"
  }
}


Now, let's evaluate our uniform sampling prompt on the validation set. If you run into rate limit issues with the API, you may want to backoff or consult one of the solutions here https://platform.openai.com/docs/guides/rate-limits/error-mitigation.

In [40]:
import tqdm

def get_predictions(model: str,
                    evaluation_examples: List[Example],
                    prompt_creation_function: Callable[[str], str],
                    **kwargv: List[str]) -> List[str]:
    """Get a list of predictions from the evaluation examples"""
    predictions = []
    for example in tqdm.tqdm(evaluation_examples):
        predicted_program = parse_example(model, example.query, prompt_creation_function, **kwargv)
        predictions.append(predicted_program)
    return predictions

def evaluate(predictions: List[str], evaluation_examples: List[Example]) -> float:
    """Evaluate the accuracy of the predictions"""
    correct = 0
    for prediction, example in zip(predictions, evaluation_examples):
        if prediction == example.program:
            correct += 1
    return correct / len(evaluation_examples)

In [41]:
random_sample_predictions = get_predictions(model="gpt-3.5-turbo",
                                            evaluation_examples=dev_examples,
                                            prompt_creation_function=random_sample_prompt,
                                            training_examples=training_examples)


100%|██████████| 40/40 [01:11<00:00,  1.79s/it]


The model should get at least 15\% exact match with randomly sampling examples.

In [48]:

# Save the predictions as `random_predictions.txt`
preds = []
with open('random_predictions.txt', 'w') as f:
    for prediction in random_sample_predictions:
        message = prediction.message.content
        answer_idx = message.find("answer") if "answer" in message else 0
        message = message[answer_idx:]
        preds.append(message)
        f.write(message)
        f.write('\n')

exact_match = evaluate(preds, dev_examples)
print(f"Exact match for uniform sampling prompt: {exact_match}")

Exact match for uniform sampling prompt: 0.325


Randomly sampling examples does not use consider the utterance when selecting examples. Next, we will try to pick examples for the prompt based on embedding similarity. First, let's install `sentence-transformers`, which we will use to get embeddings of the utterance.

In [49]:
%%capture
%pip install -U sentence-transformers

Now, let's construct embeddings using a small pretrained model for each example in our training data.

In [50]:

from sentence_transformers import SentenceTransformer, util
import torch

def get_corpus(model_name: str, examples: List[Example]) -> torch.Tensor:
    """Return a tensor of the corpus embeddings, the size of the returned tensor should be (num_examples, embedding_size)"""
    model = SentenceTransformer(model_name)
    corpus = [example.query for example in examples]
    corpus_embeddings = model.encode(corpus)
    return corpus_embeddings

corpus_embeddings = get_corpus('all-MiniLM-L6-v2', training_examples)


With the embeddings of all the training data, we can construct a function that takes in an utterance and outputs a prompt with examples with highest cosine similarity with the utterance.

In [ ]:
def get_nearest_neighbor_prompt(utterance: str,
                                training_examples: List[Example],
                                embedding_model: str,
                                corpus_embeddings: torch.Tensor,
                                num_samples: int = 10) -> str:
    model = SentenceTransformer(embedding_model)
    query_embedding = model.encode(utterance)
    cos_similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    top_results = torch.topk(cos_similarities, k=min(num_samples, len(training_examples))).indices.tolist()

    prompt = "Given the following examples of utterances and their semantic parses:\n"
    for idx in top_results:
        example = training_examples[idx]
        prompt += f"- Utterance: {example.query}\n  Parse: {example.program}\n"
    
    prompt += f"\nTranslate the following into a semantic parse: {utterance}"
    return prompt



Evaluate the similarity based on prompt on the validation data and save your predictions as `similarity_predictions.txt` with one predictions per line of the validation set. With similarity based example selection, we should get at least 20\% exact match on the validation set. Note that there could be some duplicates in the training data because we are working with a version of the data where the location names are normalized to be variables like `m0`.

In [ ]:
similarity_predictions = get_predictions(model="gpt-3.5-turbo",
                                         evaluation_examples=dev_examples,
                                         prompt_creation_function=get_nearest_neighbor_prompt,
                                         training_examples=training_examples,
                                         embedding_model='all-MiniLM-L6-v2',
                                         corpus_embeddings=corpus_embeddings)

In [57]:

preds = []
# Save the predictions as `similarity_predictions.txt`
with open('similarity_predictions.txt', 'w') as f:
    for prediction in similarity_predictions:
        message = prediction.message.content
        answer_idx = message.find("answer") if "answer" in message else 0
        message = message[answer_idx:]
        preds.append(message)
        f.write(message)
        f.write('\n')

exact_match = evaluate(preds, dev_examples)
print(f"Exact match for nearest neighbor prompt: {exact_match}")

Exact match for nearest neighbor prompt: 0.55


Let's try to improve on nearest neighbor based example search. This part will be more open ended. We will now implement a different example selection method that improves over uniform random selection. You may implement an algorithm `Diverse Demonstrations Improve In-context Compositional Generalization` (https://arxiv.org/abs/2212.06800) or come up with your own example selection method. In the report, describe the algorithm that you implemented and intuitiion of why it may it could be effective.

In [129]:
# Implement cover-utt
from collections import defaultdict
import copy
from functools import reduce


def construct_diversity_prompt(utterance: str,
                               training_examples: List[Example],
                               num_samples: int = 10) -> str:
    words = list(map(lambda x: x.lower(), utterance.split()))
    sorted_words = sorted(words, key=lambda x: len(x), reverse=True)
    # also input two word phrases into sorted words
    sorted_words += [words[i] + " " + words[i + 1] for i in range(len(words) - 1)] # 0.45
    sorted_words += [words[i] + " " + words[i + 1] + " " + words[i + 2] for i in range(len(words) - 2)] # 0.525
    sorted_words += [words[i] + " " + words[i + 1] + " " + words[i + 2] + " " + words[i + 3] for i in range(len(words) - 3)] # 0.625
    # sorted_words += [words[i] + " " + words[i + 1] + " " + words[i + 2] + " " + words[i + 3] + " " + words[i + 4] for i in range(len(words) - 4)]
    sorted_words = sorted(sorted_words, key=lambda x: len(x), reverse=True)

    training_examples = copy.deepcopy(training_examples)
    prompt = "Given the following examples of utterances and their semantic parses:\n"
    chosen_examples = [[]]
    for i, word in enumerate(sorted_words):
        if len(chosen_examples[-1]) > 0:
            chosen_examples.append([])
        for example in training_examples:
            if word in example.query.lower():
                chosen_examples[-1].append(example)
        # remove added examples in the training example list
        for example in chosen_examples[-1]:
            training_examples.remove(example)
        # print(chosen_examples)
    if chosen_examples[-1] == []:
        chosen_examples.pop(-1)
    
    chosen_examples = [sorted(examples,
                              key=lambda x: len(x.program))
                        for examples in chosen_examples]
    prompt_examples = []
    # while len(prompt_examples) < num_samples or len(chosen_examples) == 0:
    for examples in chosen_examples:
        prompt_examples.append(random.choice(examples))
    flatten = lambda l: reduce(lambda a, b: a + b, l)
    if num_samples - len(prompt_examples) > 0:
        prompt_examples += random.sample(flatten(chosen_examples), k=min(num_samples - len(prompt_examples), len(training_examples)))

    for example in prompt_examples:
        prompt += f"- Utterance: {example.query}\n  Parse: {example.program}\n"
    

    
    # for example in chosen_examples:
    #     prompt += f"- Utterance: {example.query}\n  Parse: {example.program}\n"
    
    prompt += f"\nTranslate the following into a semantic parse: - Utterance: {utterance}"
    return prompt
    

print(construct_diversity_prompt(utterance="which state borders the most states", training_examples=training_examples))


Given the following examples of utterances and their semantic parses:
- Utterance: which state borders m0
  Parse: answer ( intersection ( state , next_to_2 ( m0 ) ) )
- Utterance: what state borders the least states
  Parse: answer ( fewest ( state , next_to_2 , state ) )
- Utterance: what state borders m0
  Parse: answer ( intersection ( state , next_to_2 ( m0 ) ) )
- Utterance: through which states does the m0 run
  Parse: answer ( intersection ( state , traverse_1 ( m0 ) ) )
- Utterance: what is the capital of the state that borders the state that borders m0
  Parse: answer ( intersection ( capital , loc_2 ( intersection ( state , next_to_2 ( intersection ( state , next_to_2 ( m0 ) ) ) ) ) ) )
- Utterance: which river traverses most states
  Parse: answer ( most ( river , traverse_2 , state ) )
- Utterance: what is the most populous state
  Parse: answer ( largest_one ( population_1 , state ) )
- Utterance: what is the largest state that borders m0
  Parse: answer ( largest ( inter

In [134]:
diversity_predictions = get_predictions(model="gpt-3.5-turbo",
                                         evaluation_examples=dev_examples,
                                         prompt_creation_function=construct_diversity_prompt,
                                         training_examples=training_examples)

100%|██████████| 40/40 [01:15<00:00,  1.90s/it]


In [135]:
# Save the predictions as `diversity_predictions.txt`
preds = []
with open('diversity_predictions.txt', 'w') as f:
    for prediction in diversity_predictions:
        message = prediction.message.content
        answer_idx = message.find("answer") if "answer" in message else 0
        message = message[answer_idx:]
        preds.append(message)
        f.write(message)
        f.write('\n')

exact_match = evaluate(preds, dev_examples)
print(f"Exact match for diversity based prompt: {exact_match}")

Exact match for diversity based prompt: 0.65


Get the predictions and submit these as `diversity_predictions.txt`, where each line is a prediction for the development set. With the improved selection, we should get least 35\% exact match on the validation set.

For the report, compare the predictions from the example selection methods using 1) uniform random sampling 2) embedding-based similarity search and 3) coverage based selection. Compare and contrast the errors and submit your analysis as `report.pdf`

* hw5.ipynb (this file; please rename to match)
* random_predictions.txt
* similarity_predictions.txt
* diversity_predictions.txt
* report.pdf